In [56]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='1.12.0',image_scope='inference',instance_type='ml.c5.4xlarge')

'520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu'

In [57]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [58]:
!docker pull 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu

1.12.0-cpu: Pulling from sagemaker-tensorflow-serving
Digest: sha256:026c2a51a2e9e258481b17c110f9c9640dc010bf71463d4bd31e925ced7c6a49
Status: Image is up to date for 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu
520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu


In [59]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

INFO:sagemaker.image_uris:Defaulting to only available Python version: py310


'763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310'

In [60]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin  763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [85]:
!docker build -t tensorflow-training:2.12.0-cpu-py310-pippo-2 -f Dockerfile.train .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (3/3)                                          docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 198B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
[+] Building 0.3s (8/8) FINISHED                                 docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 198B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

In [107]:
!docker images

REPOSITORY            TAG                        IMAGE ID       CREATED         SIZE
tensorflow-training   2.12.0-cpu-py310-pippo-2   5840b74ec36d   7 minutes ago   6.13GB


In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='tensorflow-training:2.12.0-cpu-py310-pippo-2',
    role=role,
    instance_count=1,
    instance_type='local',
    hyperparameters=hyperparameters,
    output_path='file://{}/data/output'.format(os.getcwd())
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('file://{}/data/input/train.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')

In [88]:
!tar -xzf data/output/output.tar.gz

In [89]:
!SM_MODEL_DIR="./data/output" python main.py

Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/UFS14/2024-09-24/main.py", line 8, in <module>
    output_dir = os.environ['SM_OUTPUT_DIR']
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/os.py", line 680, in __getitem__
    raise KeyError(key) from None
KeyError: 'SM_OUTPUT_DIR'


In [90]:
!tar -xzf ./data/output/model.tar.gz

In [105]:
%%sh
# Specify an image name
image_name=tensorflow-training-2
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pippo-2"
echo "fullname: ${fullname} ######################"

aws ecr create-repository --repository-name "${image_name}"

aws ecr describe-repositories --repository-names "${image_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}" > /dev/null
fi

image_name: tensorflow-training-2 ######################
account: 523094669511 ######################
region: us-east-1 ######################
fullname: 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-pippo-2 ######################
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:523094669511:repository/tensorflow-training-2",
        "registryId": "523094669511",
        "repositoryName": "tensorflow-training-2",
        "repositoryUri": "523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2",
        "createdAt": 1729264018.085,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [106]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [109]:
!docker tag tensorflow-training:2.12.0-cpu-py310-pippo-2 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-pippo-2

In [118]:
!docker push 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-pippo-2

The push refers to repository [523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2]

68924abe: Preparing 
22878ecf: Preparing 
db4f6696: Preparing 
f529ae49: Preparing 
75f13e59: Preparing 
0c6c24f9: Preparing 
a1e64371: Preparing 
29631b13: Preparing 
3f03efd0: Preparing 
9b75d5ee: Preparing 
370de786: Preparing 
a16f1a7a: Preparing 
fc8740d1: Preparing 
292691fc: Preparing 
6672bf9d: Preparing 
607be3da: Preparing 
afc9b360: Preparing 
f934e4df: Preparing 
80a2a6eb: Preparing 
a0538fa4: Preparing 
6d109c54: Preparing 
f0a068df: Preparing 
443b7673: Preparing 
37a36861: Preparing 
ecb4bd81: Preparing 
34b60c81: Preparing 
07d5d0ab: Preparing 
e6c8e003: Preparing 
ab822c1c: Preparing 
3609315f: Preparing 
16f1a7a: Pushed   2.248GB/2.217GB0APushing   29.7MB/288.1MBPushing  365.8MB/502.1MBPushing  979.1MB/2.217GBPushing  1.519GB/2.217GBPushing  2.117GB/2.217GB2.12.0-cpu-py310-pippo-2: digest: sha256:bba0b52d777426a3addacb2647326ce0e3ae703b1de3e6358ec9abc4720c0921 size: 683